In [56]:
import numpy as np
import pandas as pd
import pickle

data_folder = 'output_tsne'

In [57]:
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [58]:
import ipywidgets as widgets
from ipywidgets import interact, interactive_output

In [61]:
def load_df(dataset_name):
    if dataset_name == "": return None
    
    in_name = '{}/tsne_{}.pkl'.format(data_folder, dataset_name)
    pkl_data = pickle.load(open(in_name, 'rb'))
    embeddeds = pkl_data['results']
    df = pd.DataFrame.from_records(embeddeds, exclude=["embedding"])
    return df

# global dataframe
g_df = load_df('MNIST-SMALL')

In [62]:
def _plot_surface(pivot_data, z_title='', chart_title=''):
    perps = pivot_data.columns
    lrs = pivot_data.transpose().columns
    data = [
        go.Surface(
            x = perps,
            y = lrs,
            z=pivot_data.as_matrix(),
        )
    ]
    layout = go.Layout(
        title=chart_title,
        autosize=True,
        width=800,
        height=800,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
        scene = dict(
            xaxis = dict(
                title='Perplexity'),
            yaxis = dict(
                title='Learning rate'),
            zaxis = dict(
                title=z_title),)   
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [69]:
def _plot_lines_by_key(key_name, key_val, use_log_x, use_log_y, show_inverse, show_neg):
    global g_df
    if g_df is None: return
    
    if key_name == 'perplexity':
        df_by_key = g_df[g_df[key_name]==key_val]
        x_data = np.log(df_by_key['learning_rate']) if use_log_x else df_by_key['learning_rate']
    else: # learning_rate
        df_by_key = g_df[g_df[key_name]==key_val]
        x_data = np.log(df_by_key['perplexity']) if use_log_x else df_by_key['perplexity']
    
    # customized functions that can be applied onto the y-values
    _inverse = lambda y: np.divide(1.0, y, out=np.zeros_like(y), where=(y != 0))
    _negative = lambda y: -y
    
    line_names = [
        # (line_name, display_name, show_by_default, can_modify)
        ('q_link', '[] -log(links) in low dim.', True, False),
        ('p_link', '[] -log(links) in high dim.', False, False),
        ('loss', '[] KL loss', True, False),
        ('auc_rnx', '[*] logRNX', True, True),
        ('cca_stress', '[*] CCA Stress', False, True),
        ('mds_isotonic', '[*] MDS Stress', False, True),
        ('pearsonr', '[*] Corr. Coef.', True, True),
        ('sammon_nlm', '[*] Sammon NLM', True, True)
    ]
    
    plot_data = []
    for line_name, display_name, show_by_default, can_modify in line_names:
        y = df_by_key[line_name]
        y_data = np.log(y) if use_log_y else y
        if can_modify:
            if show_inverse: y_data = _inverse(y_data)
            if show_neg: y_data = _negative(y_data)
            
        trace_i = go.Scatter(
            x = x_data,
            y = y_data,
            name = display_name,
            mode = 'lines+markers',
            line = {'shape': 'spline'},
            visible= '' if show_by_default else 'legendonly'
        )
        plot_data.append(trace_i)
    
    
    layout = dict(title = 'Metric values / NLL [{}  = {}]'.format(key_name, key_val),
      xaxis = dict(title = '{} {}'.format(
          'Log of ' if use_log_x else '',
          'Learning rate' if key_name == 'perplexity' else 'Perplexity')),
      yaxis = dict(title = '{} {}'.format(
          'Log of ' if use_log_y else '', 'Metrics value / negative LL')),
      )
        
    fig = go.Figure(data=plot_data, layout=layout)
    iplot(fig)

In [70]:
# UI controls

datasetX = widgets.Dropdown(
    options={
        "Select dataset": "",
        "MNIST mini": "MNIST-SMALL",
        "COIL-20": "COIL20",
        "Country Indicators 1999": "COUNTRY1999",
        "Country Indicators 2013": "COUNTRY2013",
        "Country Indicators 2014": "COUNTRY2014",
        "Country Indicators 2015": "COUNTRY2015",
        "Cars and Trucks 2004": "CARS04",
        "Breast Cancer Wisconsin (Diagnostic)": "BREAST-CANCER95",
        "Pima Indians Diabetes": "DIABETES",
        "Multidimensional Poverty Measures": "MPI"
    },
    value='',
    description='Dataset:',
)

pivotKeyX = widgets.Dropdown(
    options={
        'NLL in LD': 'q_link',
        'NLL in HD': 'p_link',
        'KL-loss': 'loss',
        'AUC R_NX': 'auc_rnx',
        'CorrCoef': 'pearsonr',
        'MDS Stress': 'mds_isotonic',
        'CCA Stress': 'cca_stress',
        'Sammon NLM': 'sammon_nlm'

    },
    value='q_link',
    description='Surface: ',
)

lrX = widgets.SelectionSlider(
    options=g_df['learning_rate'].unique(),
    value=100.0,
    description='LR ($\lambda$): '
)

perpX = widgets.SelectionSlider(
    options=g_df['perplexity'].unique(),
    value=50.0,
    description='Perplexity: '
)

useLogXAxisX = widgets.Checkbox(
    description='Log scale in xAxis',
    value=False,
)

useLogYAxisX = widgets.Checkbox(
    description='Log scale in yAxis',
    value=False,
)

showNegativeX = widgets.Checkbox(
    description='Negative stress func. [*]',
    value=False,
)

showInverseX = widgets.Checkbox(
    description='Inverse stress func. [*]',
    value=False,
)

In [71]:
# interactive plot

@interact(dataset_name=datasetX)
def select_dataset(dataset_name):   
    global g_df
    g_df = load_df(dataset_name)
    
    @interact(key_to_pivot=pivotKeyX)
    def plot_surface_by_name(key_to_pivot='q_link'):
        global g_df
        if g_df is None: return

        pivot_df = g_df[['learning_rate', 'perplexity', key_to_pivot]].pivot(
            index='learning_rate', columns='perplexity', values=key_to_pivot)

        _plot_surface(pivot_data=pivot_df, chart_title=key_to_pivot)

A Jupyter Widget

In [72]:
def plot_lines_by_key(dataset_name, lr, perp, use_log_x, use_log_y, show_inverse, show_neg):
    global g_df
    g_df = load_df(dataset_name)
    
    _plot_lines_by_key('learning_rate', lr, use_log_x, use_log_y, show_inverse, show_neg)
    _plot_lines_by_key('perplexity', perp, use_log_x, use_log_y, show_inverse, show_neg)
    
ui = widgets.VBox([
    widgets.HBox([datasetX, lrX, perpX]),
    widgets.HBox([useLogXAxisX, useLogYAxisX, showInverseX, showNegativeX])
])

out = widgets.interactive_output(plot_lines_by_key, 
               {'dataset_name': datasetX,
                'lr':lrX,
                'perp':perpX,
                'use_log_x': useLogXAxisX,
                'use_log_y': useLogYAxisX,
                'show_inverse': showInverseX,
                'show_neg': showNegativeX})

display(ui, out)

A Jupyter Widget

A Jupyter Widget

In [67]:
# plot running time and the number of iterations

def _plot_runningtime(key_name, key_val):
    global g_df
    if g_df is None: return
    
    if key_name == 'perplexity':
        df_by_key = g_df[g_df[key_name]==key_val]
        x_data = df_by_key['learning_rate']
    else: # learning_rate
        df_by_key = g_df[g_df[key_name]==key_val]
        x_data = df_by_key['perplexity']
    
    trace1 = go.Scatter(
        x=x_data,
        y=df_by_key['running_time'],
        name='Runnning time'
    )
    trace2 = go.Bar(
        x=x_data,
        y=df_by_key['n_iter'],
        name='# of iterations',
        yaxis='y2'
    )
    data = [trace1, trace2]
    layout = go.Layout(
        title='Running time and number of iterations [{} = {}]'.format(key_name, key_val),
        xaxis = dict(title = '{}'.format('Learning rate' if key_name == 'perplexity' else 'Perplexity')),
        yaxis=dict(
            title='Running time (s)'
        ),
        yaxis2=dict(
            title='Number of iterations',
            titlefont=dict(
                color='orange'
            ),
            tickfont=dict(
                color='orange'
            ),
            overlaying='y',
            side='right'
        ),
        legend=dict(orientation="h")
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [68]:
def plot_runningtime(dataset_name, lr, perp):
    global g_df
    g_df = load_df(dataset_name)
    
    _plot_runningtime('learning_rate', lr)
    _plot_runningtime('perplexity', perp)
    
ui2 = widgets.VBox([
    widgets.HBox([datasetX, lrX, perpX]),
])

out2 = widgets.interactive_output(plot_runningtime, 
               {'dataset_name': datasetX,
                'lr':lrX,
                'perp':perpX})

display(ui2, out2)

A Jupyter Widget

A Jupyter Widget